# Structural keys

Last time, we dealt with substructures and substructure-based filters. An extension of this concept is assembling a set of several such substructures, which can be searched as a whole against any molecule. As the normal substructure search returns binary results (True or False whether the substructure is contained within the given molecule), searching for a set of substructures within a given molecule produces a set of binary values. This set of binary values can be assembled into a binary vector, where each substructure is represented by a corresponding single bit within the vector. This defined set of substructures, called a structural key or substructure key, allows to characterize any given molecule by a binary vector. Structural keys are basically premade substructure searches that can be used to accelerate structure-related database queries, to quickly compare chemical structures to each other, and to map the feature space of a set of structures, among many other things. More details on structural keys and their applied use are available in the [daylight documentation](https://www.daylight.com/dayhtml/doc/theory/theory.finger.html). Pretty much any computational method that works on binary vectors can be used on structural keys. Let's make a structural key of our own:

In [1]:
from rdkit.Chem import AllChem as Chem
from rdkit.Chem import Draw
from rdkit.Chem.Draw import IPythonConsole
import csv

RDKit WARNING: [13:02:07] Enabling RDKit 2019.09.3 jupyter extensions


In [2]:
# load your own set, and DrugBank :)
with open('../data/chembl_mtor_ic50.csv', 'r') as csvfile:
    reader = csv.DictReader(csvfile, delimiter=";")
    mtor_ligands = [Chem.MolFromSmiles(m['Smiles']) for m in reader]

suppl = Chem.SDMolSupplier('../data/drugbank.sdf')
drugs = [m for m in suppl if m]

# let's assemble a structural key from several substructures that were introduced in the previous exercise 5:

In [3]:
ethanol_pattern = Chem.MolFromSmarts('CCO')
propanol_pattern = Chem.MolFromSmarts('CCCO')
cooh_pattern = Chem.MolFromSmarts('C(=O)[O;h1]')
salicylic_acid_pattern = Chem.MolFromSmarts('c1ccc(c(c1)C(=O)O)O')

So, we have a list of 4 substructures. We can search them all in a molecule, and produce a binary vector of 4 boolean values indicating which substructures are contained in the molecule. Let's characterize the structures in DrugBank using this amazing 4-bit key...

In [4]:
custom_key = [ethanol_pattern, propanol_pattern, cooh_pattern, salicylic_acid_pattern]
mtor_ligands_keys = [[m.HasSubstructMatch(substruct) for substruct in custom_key] for m in mtor_ligands]
len(mtor_ligands_keys), mtor_ligands_keys

(4596,
 [[False, False, False, False],
  [False, False, False, False],
  [False, False, False, False],
  [False, False, False, False],
  [False, False, False, False],
  [False, False, False, False],
  [False, False, False, False],
  [True, True, False, False],
  [True, False, False, False],
  [False, False, False, False],
  [True, True, False, False],
  [True, True, False, False],
  [False, False, False, False],
  [False, False, False, False],
  [False, False, False, False],
  [False, False, False, False],
  [False, False, False, False],
  [True, True, False, False],
  [True, False, False, False],
  [True, False, False, False],
  [False, False, False, False],
  [False, False, False, False],
  [False, False, False, False],
  [True, True, False, False],
  [False, False, False, False],
  [False, False, False, False],
  [False, False, False, False],
  [False, False, False, False],
  [False, False, False, False],
  [False, False, False, False],
  [False, False, False, False],
  [True, False

As shown above, every molecule in our ligand set is represented by four bits that correspond to the presence of our four custom-defined substructures. This is of course a toy example, and structural keys are often hundreds of bits long. However, you don't have to manually define hundreds of substructures everytime you want to use structural keys. There are several already established sets of substructure features that you can use if you do not have a specific substructures in mind, and want a general characterization of your structure set. You can always extend those generic keys by whatever explicit groups you want to focus on.

# MACCS key
is an often-used structural key consisting of 166 defined [substructures of interest](https://github.com/openbabel/openbabel/blob/master/data/MACCS.txt), also implemented in RDKit. Let's make use of it to characterize our known ligands and the DrugBank database:

In [80]:
from rdkit.Chem import MACCSkeys

In [81]:
mtor_maccs = [MACCSkeys.GenMACCSKeys(m) for m in mtor_ligands]
drugbank_maccs = [MACCSkeys.GenMACCSKeys(m) for m in drugs]
mtor_maccs[0]

As with the custom substructure keys, each molecule is represented by a vector of binary values. In the RDKit implementation, the binary vector is implemented as an RDKit object has some nice [inbuilt binary vector operations](https://www.rdkit.org/docs/source/rdkit.DataStructs.cDataStructs.html):

In [83]:
mtor_maccs[0].GetNumBits() # get size of the vector

167

In [84]:
list(mtor_maccs[0].GetOnBits()) # get indices of the bits that are set to True within the fingerprint

[25,
 32,
 33,
 36,
 42,
 47,
 51,
 55,
 58,
 59,
 60,
 61,
 62,
 64,
 65,
 67,
 69,
 73,
 77,
 80,
 81,
 83,
 87,
 88,
 92,
 94,
 96,
 97,
 98,
 101,
 102,
 105,
 106,
 107,
 110,
 112,
 117,
 120,
 121,
 124,
 125,
 130,
 131,
 133,
 134,
 135,
 136,
 137,
 142,
 144,
 145,
 146,
 148,
 150,
 151,
 154,
 156,
 158,
 159,
 160,
 161,
 162,
 163,
 164,
 165]

In [9]:
mtor_maccs[0].GetBit(25), mtor_maccs[0].GetBit(26) # getting individual bits

(True, False)

In [10]:
mtor_maccs[0].ToBitString() # write out the bit values in a string

'00000000000000000000000001000000110010000010000100010001001111101101010001000100110100011000101011100110011100101000010011001100001101111100001011101011001010111111110'

... among many other methods. For now, let's use the MACCS substructure keys to compare the relative amounts of substructures within our set of ligands and the DrugBank contents:

In [11]:
mtor_ligands_maccs_sums = [0]*mtor_maccs[0].GetNumBits() # a list of zeros of a given length
for key in mtor_maccs:
    for onbit in key.GetOnBits():
        mtor_ligands_maccs_sums[onbit] += 1
mtor_ligands_maccs_sums

[0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 68,
 0,
 0,
 115,
 0,
 37,
 0,
 0,
 0,
 907,
 0,
 468,
 0,
 0,
 124,
 216,
 52,
 1604,
 47,
 2,
 11,
 5,
 1,
 0,
 406,
 429,
 12,
 0,
 278,
 1215,
 3884,
 0,
 8,
 119,
 1257,
 910,
 0,
 47,
 26,
 129,
 5,
 24,
 89,
 616,
 974,
 343,
 135,
 625,
 11,
 2528,
 631,
 303,
 631,
 639,
 4451,
 11,
 528,
 4498,
 764,
 639,
 2,
 645,
 56,
 93,
 390,
 633,
 1024,
 3781,
 185,
 4326,
 34,
 2815,
 4293,
 727,
 925,
 2524,
 1809,
 3758,
 3365,
 1404,
 860,
 855,
 681,
 704,
 2203,
 2289,
 1434,
 2626,
 2598,
 1742,
 4532,
 196,
 3490,
 3088,
 733,
 453,
 738,
 3226,
 2851,
 1600,
 1667,
 2395,
 3027,
 3532,
 1479,
 1915,
 1270,
 1557,
 1836,
 3192,
 3473,
 47,
 4565,
 4566,
 4063,
 394,
 1639,
 4520,
 1435,
 2260,
 3292,
 3417,
 859,
 2577,
 2622,
 4290,
 1616,
 4284,
 1314,
 4595,
 3397,
 1012,
 1079,
 1484,
 4518,
 2260,
 3239,
 4569,
 2098,
 3497,
 4189,
 2825,
 3590,
 3916,
 1899,
 3689,
 2774,
 2555,
 4557,
 3435,
 4545,
 3335,
 3911,
 4591,
 4541,
 4595,
 430

In [12]:
drugbank_maccs_sums = [0]*drugbank_maccs[0].GetNumBits() # a list of zeros of a given length
for key in drugbank_maccs:
    for onbit in key.GetOnBits():
        drugbank_maccs_sums[onbit] += 1
drugbank_maccs_sums

[0,
 0,
 0,
 30,
 0,
 1,
 11,
 24,
 99,
 46,
 22,
 117,
 30,
 60,
 33,
 11,
 52,
 77,
 68,
 282,
 7,
 21,
 181,
 212,
 435,
 564,
 224,
 96,
 149,
 726,
 126,
 21,
 431,
 481,
 153,
 44,
 625,
 521,
 1312,
 153,
 167,
 166,
 698,
 1198,
 266,
 246,
 194,
 455,
 869,
 713,
 646,
 601,
 553,
 1895,
 2246,
 663,
 178,
 1633,
 669,
 604,
 685,
 695,
 1753,
 207,
 624,
 2507,
 1118,
 782,
 133,
 1551,
 295,
 526,
 2196,
 755,
 1184,
 2092,
 797,
 2245,
 703,
 2263,
 2296,
 1228,
 2080,
 2781,
 2109,
 2498,
 1685,
 1310,
 1596,
 2474,
 3533,
 3342,
 2825,
 1650,
 1481,
 3191,
 3098,
 2783,
 3056,
 1145,
 2845,
 2775,
 1934,
 786,
 3210,
 2939,
 2665,
 1534,
 1732,
 2372,
 3113,
 3377,
 2725,
 2292,
 1097,
 1828,
 1896,
 3354,
 3347,
 924,
 3223,
 3762,
 2973,
 2791,
 2463,
 3448,
 2547,
 3821,
 2689,
 2850,
 1791,
 4638,
 3554,
 2717,
 1637,
 2198,
 3264,
 4492,
 2581,
 4291,
 3262,
 1352,
 4169,
 3821,
 2802,
 3698,
 4375,
 3396,
 3793,
 2507,
 3939,
 4407,
 3963,
 4550,
 4670,
 4851,
 5254

Since the sizes of our ligand set and the size of the DrugBank database are different, it might be a good idea to divide the raw incidence counts by the total set size, thus getting a number between 0 (never appears in any structure in a set) and 1 (always appears in every structure in a set) for both the ligand and the DrugBank set:

In [13]:
mtor_ligands_maccs_scaled = [x/len(mtor_maccs) for x in mtor_ligands_maccs_sums]
mtor_ligands_maccs_scaled

[0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.014795474325500435,
 0.0,
 0.0,
 0.02502175805047868,
 0.0,
 0.00805047867711053,
 0.0,
 0.0,
 0.0,
 0.1973455178416014,
 0.0,
 0.10182767624020887,
 0.0,
 0.0,
 0.026979982593559618,
 0.04699738903394256,
 0.011314186248912098,
 0.34899912967798086,
 0.010226283724978242,
 0.0004351610095735422,
 0.002393385552654482,
 0.0010879025239338555,
 0.0002175805047867711,
 0.0,
 0.08833768494342907,
 0.0933420365535248,
 0.0026109660574412533,
 0.0,
 0.060487380330722366,
 0.2643603133159269,
 0.845082680591819,
 0.0,
 0.0017406440382941688,
 0.02589208006962576,
 0.2734986945169713,
 0.19799825935596171,
 0.0,
 0.010226283724978242,
 0.005657093124456049,
 0.028067885117493474,
 0.0010879025239338555,
 0.005221932114882507,
 0.019364664926022627,
 0.134029590948651,
 0.21192341166231507,
 0.07463011314186249,
 0.0293733681462141,
 0.13598781549173194,
 0.002393385552654482,
 0.5500435161009574,
 0.13729329852045258,
 0.06592689295039164,
 0

In [14]:
drugbank_maccs_scaled = [x/len(drugbank_maccs) for x in drugbank_maccs_sums]
drugbank_maccs_scaled

[0.0,
 0.0,
 0.0,
 0.004217629692113032,
 0.0,
 0.00014058765640376775,
 0.0015464642204414453,
 0.003374103753690426,
 0.013918177983973008,
 0.006467032194573317,
 0.0030929284408828905,
 0.016448755799240825,
 0.004217629692113032,
 0.008435259384226065,
 0.004639392661324336,
 0.0015464642204414453,
 0.007310558132995923,
 0.010825249543090117,
 0.009559960635456208,
 0.0396457191058625,
 0.0009841135948263741,
 0.0029523407844791226,
 0.025446365809081963,
 0.029804583157598763,
 0.06115563053563897,
 0.079291438211725,
 0.03149163503444397,
 0.013496415014761703,
 0.020947560804161394,
 0.10206663854913539,
 0.017714044706874738,
 0.0029523407844791226,
 0.0605932799100239,
 0.0676226627302123,
 0.021509911429776464,
 0.006185856881765781,
 0.08786728525235485,
 0.07324616898636299,
 0.1844510052017433,
 0.021509911429776464,
 0.023478138619429215,
 0.023337550963025446,
 0.0981301841698299,
 0.16842401237171375,
 0.03739631660340222,
 0.03458456347532687,
 0.027274005342330942,


So, we now have the occurrence ratio of each MACCS substructure within our set, and within DrugBank. We can now subtract the two and have a look at the differences:

In [15]:
# compute the differences, store bit numbers prior to sorting
mtor_drugbank_differences = [(i, a_b[0] - a_b[1])
                             for i, a_b in enumerate(zip(mtor_ligands_maccs_scaled, drugbank_maccs_scaled))]
mtor_drugbank_differences

[(0, 0.0),
 (1, 0.0),
 (2, 0.0),
 (3, -0.004217629692113032),
 (4, 0.0),
 (5, -0.00014058765640376775),
 (6, -0.0015464642204414453),
 (7, -0.003374103753690426),
 (8, 0.0008772963415274275),
 (9, -0.006467032194573317),
 (10, -0.0030929284408828905),
 (11, 0.008573002251237853),
 (12, -0.004217629692113032),
 (13, -0.000384780707115534),
 (14, -0.004639392661324336),
 (15, -0.0015464642204414453),
 (16, -0.007310558132995923),
 (17, 0.1865202682985113),
 (18, -0.009559960635456208),
 (19, 0.06218195713434637),
 (20, -0.0009841135948263741),
 (21, -0.0029523407844791226),
 (22, 0.0015336167844776545),
 (23, 0.017192805876343795),
 (24, -0.04984144428672688),
 (25, 0.26970769146625584),
 (26, -0.021265351309465733),
 (27, -0.013061254005188162),
 (28, -0.01855417525150691),
 (29, -0.10097873602520153),
 (30, -0.017496464202087968),
 (31, -0.0029523407844791226),
 (32, 0.027744405033405167),
 (33, 0.025719373823312505),
 (34, -0.01889894537233521),
 (35, -0.006185856881765781),
 (36, -0.

In [16]:
# let's sort the bits by the difference in MACCS incidence between our ligand set and the DrugBank database
mtor_drugbank_differences.sort(key=lambda x: x[1])
mtor_drugbank_differences

[(139, -0.3830701627843551),
 (90, -0.3485238663147203),
 (91, -0.31666727233150493),
 (123, -0.306653430136928),
 (104, -0.2907119645234574),
 (54, -0.28638650813664823),
 (72, -0.2238740965958333),
 (140, -0.22382757052416438),
 (53, -0.19178349574327738),
 (136, -0.1729773272120807),
 (89, -0.1617825303502321),
 (146, -0.1585870977238381),
 (152, -0.14396350373805333),
 (155, -0.12607253148447717),
 (48, -0.12108277089094031),
 (119, -0.11967671079210315),
 (99, -0.11832708764410693),
 (102, -0.11241001747618362),
 (29, -0.10097873602520153),
 (49, -0.0950170669010039),
 (78, -0.09143538528909852),
 (131, -0.09134058956516566),
 (82, -0.09116035839207365),
 (69, -0.0777120294947764),
 (159, -0.07220396662750028),
 (76, -0.07179596876825024),
 (50, -0.07145496111081134),
 (130, -0.06489083900731166),
 (112, -0.061299797120632604),
 (71, -0.053714120323212114),
 (154, -0.05297603512709237),
 (24, -0.04984144428672688),
 (126, -0.04584873649137994),
 (127, -0.0454534943006939),
 (143, 

The MACCS bits that are least prevalent in our MTOR ligand set compared to the DrugBank database contents are 139, 90, 91, 123, 104, 54, etc. These bits correspond to structural patterns [in the MACCS key definition](https://github.com/openbabel/openbabel/blob/master/data/MACCS.txt) "OH", "QHAACH2A" (heteroatom with one H on it, anything, anything, CH2, anything), "QHAAACH2A", "OCO", "QHACH2A" and "QHAAQH", respectively. The most prevalent MACCS bits in our set of MTOR ligands compared to the DrugBank baseline are 62, 38, 65, 77, 135 and 80. These correspond to structural patterns "A\\$!A\\$A" (any atom, cyclic bond, any atom, acyclic bond, any atom, cyclic bond, any atom), "NC(C)N", "C%N" (C, aromatic bond, N), "NAN", "Nnot%A%A" (N, nonaromatic bond, any atom, aromatic bond, any atom) and "NAAAN". Apparently, our MTOR ligand set has, compared to DrugBank, a lot more going on regarding heteroatoms (especially N), aromaticity, and rings in general. How about your set?

# What to do
 - Familiarize yourself with structural keys, if you haven't already in the lectures :) Some resources are Daylight Theory [here](https://www.daylight.com/dayhtml/doc/theory/theory.finger.html) and Dalke's website [here](http://www.dalkescientific.com/writings/NBN/fingerprints.html)
 - Try to make a small structural key of your own, and characterize your set of ligands using this key
 - Have a look at the MACCS key, characterize both your structure set and DrugBank using MACCS key
 - Which MACCS bits are significantly more/less common in your ligand set in comparison to DrugBank?
 - What substructures do these bits [correspond to](https://github.com/openbabel/openbabel/blob/master/data/MACCS.txt)? Some additional materials on how to interpret SMARTS are [here](https://www.daylight.com/dayhtml/doc/theory/theory.smarts.html) and [here](https://www.daylight.com/dayhtml_tutorials/languages/smarts/smarts_examples.html).

In [72]:
chlorbenzene = Chem.MolFromSmiles('[Cl]c1ccccc1')
cycloheptan = Chem.MolFromSmiles('C1CCCCCC1')
cyclizine_substr = Chem.MolFromSmiles('N1CCNCC1')
pyridine = Chem.MolFromSmiles('n1ccccc1')

In [73]:
import pandas as pd
h1_df = pd.read_csv('H1.csv', delimiter=';')
h1_df = h1_df[h1_df['Smiles'].isna() == False]
h1_ligands = list()

# Get RDKit Mol instances
for i in h1_df['Smiles']:
    h1_ligands.append(Chem.MolFromSmiles(i))

In [79]:
custom_key_h1 = [chlorbenzene, cycloheptan, cyclizine_substr, pyridine]
h1_ligands_keys = [[m.HasSubstructMatch(substruct) for substruct in custom_key_h1] for m in h1_ligands]

cnt = 0
for ligand in h1_ligands_keys:
    if(True in ligand):
        cnt += 1 

print(str(cnt) + ' ligands in my set contain at least one of the substructres specified in custom key.')

406 ligands in my set contain at least one of the substructres specified in custom key.


In [82]:
h1_maccs = [MACCSkeys.GenMACCSKeys(m) for m in h1_ligands]

In [92]:
print(len(list(h1_maccs[556].GetOnBits())))

68


Randomly picked ligand from my set contains 68 of MACCS substructures.

In [94]:
h1_ligands_maccs_sums = [0]*h1_maccs[0].GetNumBits() 
for key in h1_maccs:
    for onbit in key.GetOnBits():
        h1_ligands_maccs_sums[onbit] += 1

In [96]:
h1_ligands_maccs_scaled = [x/len(h1_maccs) for x in h1_ligands_maccs_sums]

In [98]:
h1_drugbank_differences = [(i, a_b[0] - a_b[1])
                             for i, a_b in enumerate(zip(h1_ligands_maccs_scaled, drugbank_maccs_scaled))]
h1_drugbank_differences.sort(key=lambda x: x[1])
h1_drugbank_differences

[(131, -0.2685387449563193),
 (151, -0.1984328922437823),
 (90, -0.18444639023463955),
 (54, -0.18125227019799445),
 (139, -0.174918959489256),
 (69, -0.15480085460185947),
 (130, -0.1533137095927269),
 (124, -0.15171175320686744),
 (53, -0.14231432946160105),
 (146, -0.1394905323949867),
 (91, -0.1367774945388618),
 (159, -0.13650588684078147),
 (84, -0.12996614077428115),
 (106, -0.12886946700619314),
 (102, -0.12217675166423034),
 (43, -0.12118622213952801),
 (140, -0.11592279587764126),
 (48, -0.10615786316667561),
 (136, -0.10099180145466014),
 (82, -0.09386347824217478),
 (29, -0.09325959291262619),
 (104, -0.090197505959217),
 (95, -0.07791865433862627),
 (109, -0.07326575445651051),
 (164, -0.07224888585126499),
 (117, -0.07121074337327304),
 (119, -0.0642504724994673),
 (132, -0.06169817057075999),
 (92, -0.060891114128474144),
 (123, -0.060114336372795674),
 (110, -0.05734513099021327),
 (154, -0.05366204956091969),
 (78, -0.052395972732072915),
 (49, -0.051399927758880797),


The MACCS bits that are least prevalent in my H1 ligand set compared to the DrugBank database contents are 131, 151, 90, 54 etc. These bits correspond to structural patterns in the MACCS key definition "QH > 1", "NH", "QHAACH2A", "QHAAQH".

The most prevalent MACCS bits in my set of H1 ligands compared to the DrugBank baseline are 86, 93, 85 and 145. These correspond to structural patterns "CH2QCH2","QCH3","CN(C)C" and "6M ring > 1".